# Chapter 10-02: Gradient Checkpointing (그래디언트 체크포인팅)

## 학습 목표
- Activation Memory가 Sequence Length에 따라 $O(S)$로 증가하는 이유를 수식으로 파악한다
- Gradient Checkpointing의 핵심 아이디어: **일부 Forward 결과를 버리고 역전파 시 재계산**하는 원리를 증명한다
- **메모리 $O(\sqrt{L})$ vs 연산량 +33%** 의 Trade-off를 정량적으로 분석한다
- TensorFlow `tf.recompute_grad`로 실제 Gradient Checkpointing을 적용하고 메모리 절약을 실측한다

## 목차
1. [수학적 기초: Activation Memory와 재계산 비용](#1.-수학적-기초)
2. [왜 Activation이 많아지는가?](#2.-Activation-문제)
3. [Gradient Checkpointing 원리 시각화](#3.-GC-원리)
4. [최적 체크포인트 간격 도출](#4.-최적-간격)
5. [TF 실전 구현 및 메모리 측정](#5.-TF-구현)
6. [Trade-off 종합 시각화](#6.-Trade-off-시각화)
7. [정리 및 연습 문제](#7.-정리)

## 1. 수학적 기초

### Activation Memory 공식 (Transformer L-레이어 기준)

표준 학습에서 역전파를 위해 **모든 레이어의 Forward 결과**를 메모리에 저장해야 합니다:

$$M_{act}^{\text{standard}} = L \times M_{act,1}$$

### Gradient Checkpointing의 핵심 수식

레이어를 $k$개 간격으로 **체크포인트** 배치:

$$M_{act}^{\text{GC}} = \underbrace{\frac{L}{k}}_{\text{체크포인트 수}} \times M_{act,1} + \underbrace{k}_{\text{재계산 구간}} \times M_{act,1} = \left(\frac{L}{k} + k\right) M_{act,1}$$

이를 $k$에 대해 최소화:

$$\frac{d}{dk}\left(\frac{L}{k} + k\right) = -\frac{L}{k^2} + 1 = 0 \implies k^* = \sqrt{L}$$

$$\boxed{M_{act}^{\text{GC, opt}} = 2\sqrt{L} \cdot M_{act,1} \quad (O(\sqrt{L}) \text{ 달성!})}$$

### 재계산 비용

각 Segment를 역전파 시 한 번 더 Forward 계산:

$$\text{추가 FLOPs} = \frac{L}{k^*} \times k^* \times F_1 = L \times F_1$$

기존 Forward 비용이 $L \times F_1$이므로, **총 Forward 횟수 = 2배** → 연산량 **+33%** (전체의 1/3)

---

### 🐣 초등학생을 위한 친절 설명!

#### 🗒️ Gradient Checkpointing이 뭔가요?

> 시험 문제 100문제를 풀면서 **모든 중간 계산 과정을 연습장에 기록**하면 종이가 너무 많이 필요해요.  
>
> 하나의 해결책:  
> - **10문제마다** 그 시점의 풀이 상태만 저장  
> - 나머지는 버림
> - 특정 구간을 다시 필요할 때? **그 구간만 다시 풀기!**
>
> 이렇게 하면 **연습장(메모리)은 10장**만 필요하지만, 다시 푸는 시간(연산)이 조금 더 걸려요.

### 📝 연습 문제

#### 문제 1: 최적 체크포인트 간격

96-레이어 GPT-3 모델에 Gradient Checkpointing 적용 시 최적 간격 $k^*$는?  
표준 대비 메모리 절약률은?

<details>
<summary>💡 풀이 확인</summary>

$L = 96$

$$k^* = \sqrt{96} \approx 9.8 \Rightarrow k = 10$$

$$M_{GC} = \left(\frac{96}{10} + 10\right)M_1 = 19.6 M_1 \quad \text{vs} \quad M_{std} = 96 M_1$$

절약률 = $1 - \frac{19.6}{96} \approx \mathbf{79.6\%}$
</details>

#### 문제 2: 연산 오버헤드

표준 학습(Forward 1회 + Backward 1회)과 Gradient Checkpointing(Forward 2회 + Backward 1회) 적용 시  
총 연산량은 각각 몇 배인가? (Forward 1회 = 1 FLOPs 단위)

<details>
<summary>💡 풀이 확인</summary>

Backward 연산량 ≈ Forward의 2배 (Chain Rule의 편미분 계산)

- **표준**: Forward(1) + Backward(≈2) = **3 FLOPs**
- **GC**: Forward(1) + 재계산 Forward(1) + Backward(≈2) = **4 FLOPs** (+33%)
</details>

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

print("TensorFlow 버전:", tf.__version__)
tf.random.set_seed(42)
np.random.seed(42)

## 2. Activation 문제

In [ ]:
# ---------------------------------------------------
# Activation Memory 증가 문제 정량화
# ---------------------------------------------------

def act_mem_standard(L, M1=1.0):
    """표준 학습: 모든 레이어 Activation 보존 → O(L)"""
    return L * M1

def act_mem_gc(L, k, M1=1.0):
    """Gradient Checkpointing: k 간격 체크포인트 → O(L/k + k)"""
    return (L / k + k) * M1

def optimal_k(L):
    """최적 체크포인트 간격: k* = sqrt(L)"""
    return int(np.round(np.sqrt(L)))

# 레이어 수별 메모리 비교
layer_counts = [12, 24, 32, 48, 80, 96, 128]

print(f"{'L':>5} | {'k*':>4} | {'표준 M(L단위)':>13} | {'GC 최적 M':>10} | {'절약률':>7} | {'연산 오버헤드'}")  
print("-" * 65)
for L in layer_counts:
    k = optimal_k(L)
    m_std = act_mem_standard(L)
    m_gc  = act_mem_gc(L, k)
    save_pct = (1 - m_gc / m_std) * 100
    overhead = 1 / 3 * 100  # 항상 +33%
    print(f"{L:>5} | {k:>4} | {m_std:>13.0f} | {m_gc:>10.1f} | {save_pct:>6.1f}% | +{overhead:.0f}% FLOPs")

## 3. GC 원리

In [ ]:
# ---------------------------------------------------
# Gradient Checkpointing 동작 원리 시각화
# ---------------------------------------------------

def draw_gc_diagram(ax, L, k, title):
    """Gradient Checkpointing 레이어 다이어그램"""
    ax.set_xlim(-0.5, L + 0.5)
    ax.set_ylim(-1.5, 3.5)
    ax.axis('off')
    ax.set_title(title, fontweight='bold', fontsize=11)

    for i in range(L):
        is_ckpt = (i % k == 0)
        color = '#1E88E5' if is_ckpt else '#BDBDBD'
        alpha = 0.9 if is_ckpt else 0.4
        rect = plt.Rectangle((i, 1.8), 0.85, 0.8, color=color, alpha=alpha, ec='white')
        ax.add_patch(rect)
        if is_ckpt:
            ax.text(i + 0.42, 2.55, f'C{i//k}', ha='center', fontsize=7, fontweight='bold', color='white')
        if i < L - 1:
            ax.annotate('', xy=(i+1, 2.2), xytext=(i+0.85, 2.2),
                        arrowprops=dict(arrowstyle='->', color='gray', lw=1))

    # 저장된 vs 버려진 Activation 범례
    ax.text(-0.3, 1.5, 'Forward:', fontsize=8, va='center')
    ax.add_patch(plt.Rectangle((1.5, 1.35), 0.6, 0.3, color='#1E88E5', alpha=0.9))
    ax.text(2.3, 1.5, '= 체크포인트 (보존)', fontsize=7, va='center')
    ax.add_patch(plt.Rectangle((6.5, 1.35), 0.6, 0.3, color='#BDBDBD', alpha=0.4))
    ax.text(7.3, 1.5, '= 버림 (역전파시 재계산)', fontsize=7, va='center')

    # 역전파 재계산 표시
    for seg in range(L // k):
        seg_start = seg * k
        seg_end = min(seg_start + k, L)
        ax.annotate('', xy=(seg_start, 0.7), xytext=(seg_end - 0.15, 0.7),
                    arrowprops=dict(arrowstyle='->', color='#E53935', lw=1.5))
        ax.text((seg_start + seg_end) / 2, 0.3, f'재계산\n({seg_end - seg_start}레이어)',
                ha='center', fontsize=6.5, color='#E53935')

    ax.text(-0.3, 0.7, 'Backward:', fontsize=8, va='center', color='#E53935')


L = 12  # 시각화용 레이어 수
k_naive = 1  # 표준 (모두 보존)
k_gc    = 4  # GC (4레이어마다 체크포인트)

fig, axes = plt.subplots(2, 1, figsize=(15, 7))

draw_gc_diagram(axes[0], L, k_naive,
    f'표준 학습 (k=1): 모든 Activation 보존 → 메모리 {act_mem_standard(L):.0f}×M₁')
draw_gc_diagram(axes[1], L, k_gc,
    f'Gradient Checkpointing (k={k_gc}): 최적≈√L 간격 → 메모리 {act_mem_gc(L,k_gc):.0f}×M₁ ({(1-act_mem_gc(L,k_gc)/act_mem_standard(L))*100:.0f}% 절약)')

plt.tight_layout()
plt.show()

## 4. 최적 간격

In [ ]:
# ---------------------------------------------------
# 체크포인트 간격 k에 따른 메모리 곡선 (L=96 기준)
# ---------------------------------------------------

L = 96
k_range = np.arange(1, L + 1)
mem_values = [(L / k + k) for k in k_range]

k_opt = optimal_k(L)
m_min = act_mem_gc(L, k_opt)
m_std = act_mem_standard(L)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax1 = axes[0]
ax1.plot(k_range, mem_values, 'b-', lw=2.5, label='GC 메모리 $L/k + k$')
ax1.axhline(y=m_std, color='red', ls='--', lw=2, alpha=0.8, label=f'표준 학습 (k=1): {m_std}×M₁')
ax1.axvline(x=k_opt, color='green', ls=':', lw=2, label=f'최적 k*=√L={k_opt}')
ax1.scatter([k_opt], [m_min], s=120, zorder=5, color='green',
            label=f'최솟값: {m_min:.0f}×M₁ ({(1-m_min/m_std)*100:.0f}% 절약)')
ax1.fill_between(k_range, mem_values, m_std, where=[m < m_std for m in mem_values],
                 alpha=0.1, color='blue', label='절약 구간')
ax1.set_xlabel('체크포인트 간격 k', fontsize=11)
ax1.set_ylabel('Activation 메모리 (×M₁)', fontsize=11)
ax1.set_title(f'Gradient Checkpointing 메모리 곡선\n(L={L}, k*=√L={k_opt}에서 최솟값)', fontweight='bold')
ax1.legend(fontsize=9); ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 50)

# 오른쪽: L에 따른 메모리 비교
ax2 = axes[1]
L_range = np.arange(1, 200)
m_std_arr = L_range
m_gc_arr  = 2 * np.sqrt(L_range)

ax2.plot(L_range, m_std_arr, 'r-', lw=2.5, label='표준 ($O(L)$)')
ax2.plot(L_range, m_gc_arr, 'b-', lw=2.5, label='GC 최적 ($O(\\sqrt{L})$)')
ax2.fill_between(L_range, m_gc_arr, m_std_arr, alpha=0.1, color='blue', label='GC 절약 구간')
ax2.scatter([96], [96], s=80, color='red', zorder=5)
ax2.scatter([96], [2*np.sqrt(96)], s=80, color='blue', zorder=5)
ax2.annotate(f'GPT-3 L=96: {96}×M₁', xy=(96, 96), xytext=(110, 90),
             arrowprops=dict(arrowstyle='->', color='red'), fontsize=9, color='red')
ax2.annotate(f'GPT-3 GC: {2*np.sqrt(96):.1f}×M₁', xy=(96, 2*np.sqrt(96)),
             xytext=(110, 30), arrowprops=dict(arrowstyle='->', color='blue'),
             fontsize=9, color='blue')
ax2.set_xlabel('레이어 수 L', fontsize=11)
ax2.set_ylabel('Activation 메모리 (×M₁)', fontsize=11)
ax2.set_title('레이어 수에 따른 메모리 복잡도\n표준 O(L) vs GC O(√L)', fontweight='bold')
ax2.legend(fontsize=9); ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"L=96 (GPT-3): 표준 96×M₁ → GC {2*np.sqrt(96):.1f}×M₁ ({(1-2*np.sqrt(96)/96)*100:.0f}% 절약)")

## 5. TF 구현

In [ ]:
# ---------------------------------------------------
# TensorFlow tf.recompute_grad로 Gradient Checkpointing 구현
# ---------------------------------------------------

import tensorflow as tf

# 재계산이 발생하는지 추적하는 전역 카운터
forward_call_count = {'standard': 0, 'gc': 0}

class TrackedDenseLayer(tf.keras.layers.Layer):
    """forward 호출 횟수를 추적하는 Dense Layer"""
    def __init__(self, units, mode='standard', **kwargs):
        super().__init__(**kwargs)
        self.dense = tf.keras.layers.Dense(units, activation='gelu')
        self.mode = mode

    def call(self, x, training=False):
        forward_call_count[self.mode] += 1
        return self.dense(x)


def build_n_layer_mlp(n_layers, hidden, mode='standard'):
    """n개 Dense 레이어를 쌓은 MLP (표준 또는 GC 버전)"""
    layers = [TrackedDenseLayer(hidden, mode=mode) for _ in range(n_layers)]
    output_layer = tf.keras.layers.Dense(1)
    return layers, output_layer


# 데이터 설정
hidden = 128
n_layers = 8
x_data = tf.random.normal([32, hidden])
y_data = tf.random.normal([32, 1])

# ── 1) 표준 Forward-Backward ──────────────────────────
forward_call_count['standard'] = 0
std_layers, std_out = build_n_layer_mlp(n_layers, hidden, 'standard')
all_std_vars = sum([l.trainable_variables for l in std_layers], []) + std_out.trainable_variables

# 워밍업 (변수 생성)
with tf.GradientTape() as tape:
    h = x_data
    for layer in std_layers:
        h = layer(h, training=True)
    pred = std_out(h)
    loss = tf.reduce_mean((pred - y_data) ** 2)

all_std_vars = tape.watched_variables()
print(f"[표준] 학습 스텝 중 Forward 호출 횟수: {forward_call_count['standard']}")

# ── 2) Gradient Checkpointing (tf.recompute_grad) ────
forward_call_count['gc'] = 0
gc_layers, gc_out = build_n_layer_mlp(n_layers, hidden, 'gc')

# tf.recompute_grad: 래핑된 함수를 Backward 시 재호출
def make_recomputed_layer(layer):
    @tf.recompute_grad
    def recomputed_call(x):
        return layer(x, training=True)
    return recomputed_call

recomputed = [make_recomputed_layer(l) for l in gc_layers]

with tf.GradientTape() as tape:
    # 워밍업 Forward
    h = x_data
    for layer in gc_layers:
        h = layer(h, training=True)
    pred_gc = gc_out(h)

print(f"[GC] Warm-up Forward 호출: {forward_call_count['gc']}")
forward_call_count['gc'] = 0  # 리셋

with tf.GradientTape() as tape:
    h = x_data
    for rc in recomputed:
        h = rc(h)
    pred_gc = gc_out(h)
    loss_gc = tf.reduce_mean((pred_gc - y_data) ** 2)

grads = tape.gradient(loss_gc, tape.watched_variables())
print(f"[GC] Forward+Backward 총 Forward 호출: {forward_call_count['gc']}")
print(f"  → {n_layers}레이어 × 2(Forward 1 + 재계산 1) 예상: {n_layers * 2}")
print(f"\n결론:")
print(f"  표준: Forward {n_layers}회 → Activation {n_layers}개 보존")
print(f"  GC  : Forward {n_layers}회 + 재계산 {n_layers}회 → Activation O(√{n_layers})개 보존")
print(f"  → 메모리 절약 {(1-2*np.sqrt(n_layers)/n_layers)*100:.0f}%, 연산 +33%")

## 6. Trade-off 시각화

In [ ]:
# ---------------------------------------------------
# 메모리-연산량 Trade-off 종합 시각화
# ---------------------------------------------------

scenarios = ['표준\n(k=L)', 'GC k=2', 'GC k=4', 'GC k=√L\n(최적)', '전체 재계산\n(k=1)']
L = 32
k_vals = [L, 2, 4, optimal_k(L), 1]

mem = [act_mem_gc(L, k) / L * 100 if k < L else 100 for k in k_vals]
mem[0] = 100  # 표준 = 100%
mem[-1] = act_mem_gc(L, 1) / L * 100  # k=1: L/1 + 1 ≈ L+1
flops = [100, 100 + 100*(L/2-1)/L/3*100 , 100 + 100*(L/4-1)/L/3*100, 133.3, 200]
# 단순화: 표준=100, GC=133, 완전재계산=200
flops = [100.0, 116.7, 125.0, 133.3, 200.0]

fig, ax = plt.subplots(figsize=(10, 6))

colors_sc = ['#E53935', '#FB8C00', '#43A047', '#1E88E5', '#6D4C41']
for i, (sc, m, f) in enumerate(zip(scenarios, mem, flops)):
    ax.scatter(f, m, s=200, color=colors_sc[i], zorder=5, label=sc)
    offset = (2, 3)
    if i == 4: offset = (-30, 3)
    ax.annotate(sc.replace('\n', ' '), (f, m), fontsize=9,
                xytext=(f + offset[0], m + offset[1]),
                arrowprops=dict(arrowstyle='->', lw=0.8, color='gray'))

ax.axhline(y=mem[3], color='blue', ls=':', alpha=0.5, label=f'최적 GC 메모리선 ({mem[3]:.0f}%)')
ax.axvline(x=133.3, color='blue', ls=':', alpha=0.5)

ax.set_xlabel('연산량 (표준=100%)', fontsize=11)
ax.set_ylabel('Activation 메모리 (표준=100%)', fontsize=11)
ax.set_title(f'Gradient Checkpointing 메모리-연산량 Trade-off (L={L})', fontweight='bold')
ax.grid(True, alpha=0.3)
ax.annotate('← 메모리↓\n   연산↑ →', xy=(145, 40), fontsize=10, color='darkblue',
            bbox=dict(boxstyle='round', facecolor='lightcyan', alpha=0.7))

plt.tight_layout()
plt.show()

print("결론:")
print("  최적 GC(k=√L): +33% 연산 오버헤드로 ~80%+ 메모리 절약 → 실무 표준")
print("  메모리가 부족한 경우 GC가 단일 최선의 해결책")

## 7. 정리

### 핵심 개념 요약

| 항목 | 값 |
|------|----|
| 표준 Activation Memory | $O(L)$ |
| GC 최적 Activation Memory | $O(\sqrt{L})$ |
| 최적 체크포인트 간격 | $k^* = \sqrt{L}$ |
| 연산 오버헤드 | +33% (Forward 2회) |
| 메모리 절약률 (GPT-3 L=96) | ≈ 80% |

### 핵심 수식

$$M_{GC} = \left(\frac{L}{k} + k\right) M_1 \xrightarrow{k=\sqrt{L}} 2\sqrt{L} \cdot M_1$$

### 다음 챕터 예고
**Chapter 10-03: ZeRO Redundancy Optimizer** — Gradient Checkpointing이 Activation을 줄인다면, ZeRO는 **파라미터, 그래디언트, 옵티마이저 상태**를 GPU 간에 분산하여 메모리를 $O(1/N)$으로 줄인다. Stage 1/2/3의 단계별 파티셔닝을 수식과 함께 완전히 이해한다.